In [1]:
import os, glob 

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

pandas = pd.read_table("lpmc02.dat")
database = db.Database("LPMC",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Exclude
# exclude = (  ArrivalTimeHours_1   ==  -1  )
# database.remove(exclude)
  
# Choice
chosenAlternative = travel_mode

#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1 = Beta('Constant1',0,None,None,1)
Constant2 = Beta('Constant2',0,None,None,0)
Constant3 = Beta('Constant3',0,None,None,0)
Constant4 = Beta('Constant4',0,None,None,0)
Cost = Beta('Cost',0,None,None,0)
Total_TT1 = Beta('Total_TT1',0,None,None,0)
Total_TT2 = Beta('Total_TT2',0,None,None,0)
Total_TT3 = Beta('Total_TT3',0,None,None,0)
Total_TT4 = Beta('Total_TT4',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

dur_public = DefineVariable('dur_public', dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int ,database)
cost_public = DefineVariable('cost_public', cost_transit ,database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

Opt1 = Constant1 + Total_TT1 * dur_walking
Opt2 = Constant2 + Total_TT2 * dur_cycling
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * dur_public
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * dur_driving
V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}

output_dir = "./model1-output"
filepath = os.path.join(output_dir, "logit_lpmc_model1_time")
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
# delete previously saved html and pickle
for file in glob.glob(f"{filepath}*"):
    os.remove(file)
    
# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = filepath
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

pandasResults

Nbr of observations: 5000
LL(0) =    -6931.472
LL(beta) = -4273.573
rho bar square = 0.382
Output file: ./model1-output/logit_lpmc_model1_time.html


,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
Constant2,-4.61,0.172,-26.8,0,0.214,-21.5,0
Constant3,-2.27,0.114,-19.8,0,0.149,-15.2,0
Constant4,-1.88,0.107,-17.6,0,0.15,-12.6,0
Cost,-0.155,0.015,-10.3,0,0.0148,-10.5,0
Total_TT1,-8.18,0.293,-27.9,0,0.453,-18,0
Total_TT2,-5.84,0.442,-13.2,0,0.506,-11.5,0
Total_TT3,-3.6,0.235,-15.3,0,0.241,-15,0
Total_TT4,-6.49,0.332,-19.6,0,0.372,-17.5,0


#### Let's compare this model with the previous one developped (Model 0)

In order to compare the two models, we take this one as the unrestricted model, and Model 0 is the restricted one. Model 0 produces 5 estimates while Model 1 produces 8. Thus we have to compare our test statistic with the chi squared ditribution with 3 degrees of freedom $\chi^2_{3}$ .

In [8]:
LLModel_0 = -4572.870
- 2 *(LLModel_0 - results.data.logLike)

598.5940731918472

- 2 *(LLModel_1 - LLModel_0) = 598.594 > $\chi^2_{3,0.99}$ = 11.344

Thus we can reject Model0 at 99% level of confidence, and we keep Model1 as our preferred model.